In [ ]:
# Download the dataset
!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz -O /content/food-101.tar.gz

# Extract the dataset
!tar -xzvf /content/food-101.tar.gz -C /content/


In [4]:
# Set data directory paths


import os
import shutil
import numpy as np

base_dir = '/content/food-101/images'
classes = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# Create train, validation, and test directories
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

    # Get all images in class directory
    images = os.listdir(os.path.join(base_dir, cls))
    np.random.shuffle(images)

    # Split images into train, val, and test
    train_images = images[:int(0.7 * len(images))]
    val_images = images[int(0.7 * len(images)):int(0.85 * len(images))]
    test_images = images[int(0.85 * len(images)):]

    # Move images to respective directories
    for img in train_images:
        shutil.move(os.path.join(base_dir, cls, img), os.path.join(train_dir, cls))

    for img in val_images:
        shutil.move(os.path.join(base_dir, cls, img), os.path.join(val_dir, cls))

    for img in test_images:
        shutil.move(os.path.join(base_dir, cls, img), os.path.join(test_dir, cls))


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [6]:


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')


Found 70700 images belonging to 101 classes.
Found 15150 images belonging to 101 classes.


In [7]:
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flattening and Dense layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))  # 101 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint


checkpoint_path = "model_checkpoint_epoch_{epoch:02d}_loss_{val_loss:.2f}.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1,
                             save_best_only=False, mode='auto', save_freq='epoch')


In [10]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # depends on your data
    epochs=20,  # adjust based on your requirement
    validation_data=val_generator,
    validation_steps=50,
    callbacks=[checkpoint]) # depends on your data


Epoch 1/20
100/100 [==============================] - ETA: 0s - loss: 4.6157 - accuracy: 0.0116
Epoch 1: saving model to model_checkpoint_epoch_01_loss_4.62.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 56s 565ms/step - loss: 4.6157 - accuracy: 0.0116 - val_loss: 4.6158 - val_accuracy: 0.0100
Epoch 2/20
100/100 [==============================] - ETA: 0s - loss: 4.6161 - accuracy: 0.0097
Epoch 2: saving model to model_checkpoint_epoch_02_loss_4.62.h5
100/100 [==============================] - 63s 634ms/step - loss: 4.6161 - accuracy: 0.0097 - val_loss: 4.6164 - val_accuracy: 0.0056
Epoch 3/20
100/100 [==============================] - ETA: 0s - loss: 4.6152 - accuracy: 0.0097
Epoch 3: saving model to model_checkpoint_epoch_03_loss_4.61.h5
100/100 [==============================] - 65s 652ms/step - loss: 4.6152 - accuracy: 0.0097 - val_loss: 4.6125 - val_accuracy: 0.0106
Epoch 4/20
100/100 [==============================] - ETA: 0s - loss: 4.6170 - accuracy: 0.0106
Epoch 4: saving model to model_checkpoint_epoch_04_loss_4.62.h5
100/100 [==============================] - 58s 580ms/step - loss: 4.6170 - accuracy: 0.0106 - val_loss: 4.6159 - val_a

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

eval_result = model.evaluate(test_generator)
print(f'Test Loss: {eval_result[0]}, Test Accuracy: {eval_result[1]}')


Found 15150 images belonging to 101 classes.
474/474 [==============================] - 63s 133ms/step - loss: 4.6155 - accuracy: 0.0099
Test Loss: 4.615548133850098, Test Accuracy: 0.009900989942252636
